<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/lecture-idea/60_linear_algebra_2/240_Eigenvalue_Jacobi_Method_numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
# This cell is for the Google Colaboratory
# https://stackoverflow.com/a/63519730
if 'google.colab' in str(get_ipython()):
  path_py = '/content/nmisp_py'

  import os
  if not os.path.exists(path_py):
    import subprocess
    subprocess.run(
        ('git', 'clone', 'https://github.com/kangwonlee/nmisp_py')
    )
  assert os.path.exists(path_py)

  import sys
  sys.path.insert(0, path_py)



In [ ]:
import os
from typing import List, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import numpy.random as nr
import numpy.testing as nt

import jacobi
import matrix



In [ ]:
Scalar = Union[int, float, complex]
Vector = List[Scalar]
Matrix = List[Vector]



The following function would visualize a numerical matrix using the [Hinton Diagram](https://matplotlib.org/stable/gallery/specialty_plots/hinton_demo.html).<br>
아래 함수는 행렬을 [힌튼 다이어그램](https://matplotlib.org/stable/gallery/specialty_plots/hinton_demo.html)으로 시각화할 것이다.



In [ ]:
def hinton(matrix, max_weight=None, ax=None):
    '''
    Draw Hinton diagram for visualizing a weight matrix.
    https://matplotlib.org/stable/gallery/specialty_plots/hinton_demo.html
    '''
    if ax is None:
      b_ax_none = True
      ax = plt.gca()
    else:
      b_ax_none = False

    if not max_weight:
        max_weight = 2 ** np.ceil(np.log2(np.abs(matrix).max()))

    ax.patch.set_facecolor('gray')
    ax.set_aspect('equal', 'box')

    ax.xaxis.set_major_locator(plt.NullLocator())
    ax.yaxis.set_major_locator(plt.NullLocator())

    for (y, x), w in np.ndenumerate(matrix):
        color = 'white' if w > 0 else 'black'
        size = np.sqrt(abs(w) / max_weight)
        rect = plt.Rectangle([x - size / 2, y - size / 2], size, size,
                             facecolor=color, edgecolor=color)
        ax.add_patch(rect)

    ax.autoscale_view()
    ax.invert_yaxis()

    if b_ax_none:
      plt.show()
      plt.close()



# 자코비 고유치 알고리듬<br>Jacobi Eigenvalue Algorithm



ref : [[0](https://en.wikipedia.org/wiki/Jacobi_eigenvalue_algorithm)], [[1](https://en.wikipedia.org/wiki/Matrix_similarity)], [[2](https://mathworld.wolfram.com/JacobiTransformation.html)]



해당 알고리듬은 대칭행렬의 모든 고유치와를 고유벡터를 한번에 구할 수 있다.<br>The algorithm can find all eigenvalues and eigenvectors of a symmetric matrix.


행렬의 *상사변환* 의 일종인 *자코비 변환* 을 반복한다.<br>It iterates the *Jacobi transformation*, a *similarity transformation*.



상사변환이란, 어떤 행렬 $A$ 에 행렬 $P$ 와 그 역행렬 $P^{-1}$을 좌우에서 곱해주는 것이다.<br>
The *similarity transformation* is to multiply a matrix $P$ and its inverse $P^{-1}$ to a matrix $A$ on both sides.



$$
B = P^{-1}AP
$$


행렬 $A$ 와 $B$ 의 고유치는 같다.<br>The matrices $A$ and $B$ have the same eigenvalues.



자코비 변환은, 행렬의 비대각 원소들 가운데 하나를 0으로 만드는 것이다..<br>Jacobi transformation is to diagonalize a two dimensional subspace in the basis of an n-dimensional space.



$3 \times 3$ 행렬 사례를 살펴 보도록 하자.<br>Let's take a look at a $3 \times 3$ matrix case.



In [ ]:
n = 3

matA = nr.randint(-5, 5, (n, n),)
matA = matA @ matA.T
matA += np.diag(nr.randint(10, 100, (n,)))



In [ ]:
hinton(matA)



오른쪽은 고유벡터, 왼쪽은 고유치를 보여 줄 것이다.<br>Right, left matrices will show the eigenvectors and eigenvalues respectively.



행렬의 열 벡터의 변화를 따라 가 보자.<br>Let's keep track of the changes of the column vectors of the matrices.



In [ ]:
w, v = jacobi.eigenvalue_algorithm(matA, b_plot=True)



아래 $W$ 행렬의 대각원소가 $A$ 행렬의 고유치이다.<br>The diagonal elements of the following matrix $W$ are the eigenvalues of the matrix $A$.



In [ ]:
w



In [ ]:
hinton(w)



아래 $V$ 행렬의 열벡터가 $A$ 행렬의 고유벡터이다.<br>The column vectors of the following matrix $V$ are the eigenvectors of the matrix $A$.



In [ ]:
v



In [ ]:
hinton(v)



## 대각화<br>Diagonalization



이렇게 구한 고유 벡터 행렬 $V$ 로 행렬 $A$ 를 상사 변환하자.<br>
Let's simiarity-transform $A$ with the eigenvector matrix $V$.



In [ ]:
vT = matrix.transpose_mat(v)



$\Lambda = V^T A V$


In [ ]:
matLambda = matrix.mul_mat(vT, matrix.mul_mat(matA, v))



In [ ]:
matLambda



In [ ]:
hinton(matLambda)



$W$ 행렬과 $\Lambda$ 행렬은 같은가?<br>Are $W$ and $\Lambda$ matrices the same?



In [ ]:
nt.assert_allclose(
    np.array(matLambda),
    np.array(w),
    atol=1e-7,
    rtol=1e-7,
)



반대의 경우는 어떤가?<br>What about the opposite case?


$V \Lambda V^T == A$ ?


In [ ]:
nt.assert_allclose(
    np.array(matrix.mul_mat(v, matrix.mul_mat(w, vT))),
    np.array(matA),
    atol=1e-7,
)



## Test



In [ ]:
# https://en.wikipedia.org/wiki/Jacobi_eigenvalue_algorithm

mat_test_A = [
    [4, -30, 60, -35],
    [-30, 300, -675, 420],
    [60, -675, 1620, -1050],
    [-35, 420, -1050, 700],
]



In [ ]:
hinton(mat_test_A)



In [ ]:
result_test_w, result_test_v = jacobi.eigenvalue_algorithm(mat_test_A)



In [ ]:
nt.assert_allclose(
    np.diag(np.array(result_test_w)),
    np.array([2585.25381092892231, 37.1014913651276582, 1.4780548447781369, 0.1666428611718905]),
)



In [ ]:
expected_v_transpose = [
    [0.0291933231647860588, -0.328712055763188997, 0.791411145833126331, -0.514552749997152907],
    [0.179186290535454826, -0.741917790628453435, 0.100228136947192199, 0.638282528193614892],
    [0.582075699497237650, -0.370502185067093058, -0.509578634501799626, -0.514048272222164294],
    [0.792608291163763585, 0.451923120901599794, 0.322416398581824992, 0.252161169688241933],
]



Let's compare two matrices side by side.<br>
두 행렬을 옆에 나란이 놓고 비교해 보자.



In [ ]:
ax1 = plt.subplot(1, 2, 1)
hinton(np.array(result_test_v).T, ax=ax1)

ax2 = plt.subplot(1, 2, 2)
hinton(expected_v_transpose, ax=ax2)



In [ ]:
for result_list, expected_list in zip(np.array(result_test_v).T.tolist(), expected_v_transpose):
    result = np.array(result_list)
    expected = np.array(expected_list)

    try:
        nt.assert_allclose(result, expected)
    except AssertionError as e:
        nt.assert_allclose(result, -expected)



### Similarity Transformation Example<br>행렬의 상사 변환 사례


An arbitrary symmetric matrix<br>
임의의 대칭 행렬



In [ ]:
a = nr.randint(-5, 5, size=(3, 3))
ata = a.T @ a
ata



Eigenvalues `v` and Eigenvectors `w` of the matrix<br>
해당 행렬의 고유치 `v`와 고유벡터 `w`



In [ ]:
import numpy.linalg as nl

w, v = nl.eig(ata)
w, v



A rotation matrix<br>
회전 행렬



In [ ]:
theta = np.deg2rad(30)
c = np.cos(theta)
s = np.sin(theta)
p = np.array((
    (c, -s, 0),
    (s, c, 0),
    (0, 0, 1),
))



Eigenvalues and eigenvectors of the rotation matrix<br>
회전 행렬의 고유치와 고유벡터



In [ ]:
wp, vp = nl.eig(p)



Eigenvalues 고유치



In [ ]:
wp



Eigenvectors 고유벡터

ref : https://stackoverflow.com/questions/37149933/how-to-set-max-output-width-in-numpy



In [ ]:
np.set_printoptions(edgeitems=30, linewidth=100,)
vp



Similarity transformation<br>
상사변환



In [ ]:
p_a_pi = p @ ata @ nl.inv(p)
wpp, vpp = nl.eig(ata)



In [ ]:
wpp



In [ ]:
vpp



Same eigenvalues?<br>
고유치가 같은가?



In [ ]:
nt.assert_allclose(wpp, w)



Same eigenvectors?<br>
고유벡터가 같은가?



In [ ]:
nt.assert_allclose(vpp, v)



#### An arbitrary matrix `p`<br>
임의의 `p` 행렬



In [ ]:
p = nr.randint(-5, 5, size=(3, 3)) + np.diag(nr.rand(3, 1))
p



Multiplying `p` and its inverse<br>
`p`와 그 역행렬을 곱함



In [ ]:
p_a_pi = p @ ata @ nl.inv(p)
wpp2, vpp2 = nl.eig(ata)



In [ ]:
nt.assert_allclose(vpp2, v)



In [ ]:
nt.assert_allclose(wpp2, w)



### $7\times7$ matrix<br>$7\times7$ 행렬



In [ ]:
n = 7

matA = nr.rand(n, n) - 0.5
matA = matA @ matA.T

w77, v77 = jacobi.eigenvalue_algorithm(matA, b_plot=True)



## Final Bell<br>마지막 종



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

